In [100]:
from collections import Counter
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
from datetime import datetime
from google.colab import drive
import os, zipfile, glob
import pickle

In [103]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# English books from the gutenberg.org for text generation (top 5 in volume with the right encoding)
'14860-0'
'11100-0'
'14140-0'
'16847-0'
'18762-0'

In [106]:
!wget "https://www.gutenberg.org/files/14860/14860-0.txt"

--2022-01-17 17:47:06--  https://www.gutenberg.org/files/14860/14860-0.txt
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1919120 (1.8M) [text/plain]
Saving to: ‘14860-0.txt.2’

14860-0.txt.2       100%[===================>]   1.83M  3.98MB/s    in 0.5s    

2022-01-17 17:47:06 (3.98 MB/s) - ‘14860-0.txt.2’ saved [1919120/1919120]



In [107]:
!wget "https://www.gutenberg.org/files/11100/11100-0.txt"

--2022-01-17 17:47:09--  https://www.gutenberg.org/files/11100/11100-0.txt
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1548962 (1.5M) [text/plain]
Saving to: ‘11100-0.txt.2’

11100-0.txt.2       100%[===================>]   1.48M  3.74MB/s    in 0.4s    

2022-01-17 17:47:10 (3.74 MB/s) - ‘11100-0.txt.2’ saved [1548962/1548962]



In [108]:
!wget "https://www.gutenberg.org/files/14140/14140-0.txt"

--2022-01-17 17:47:11--  https://www.gutenberg.org/files/14140/14140-0.txt
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1379661 (1.3M) [text/plain]
Saving to: ‘14140-0.txt.2’

14140-0.txt.2       100%[===================>]   1.32M  3.33MB/s    in 0.4s    

2022-01-17 17:47:12 (3.33 MB/s) - ‘14140-0.txt.2’ saved [1379661/1379661]



In [109]:
!wget "https://www.gutenberg.org/files/16847/16847-0.txt"

--2022-01-17 17:47:14--  https://www.gutenberg.org/files/16847/16847-0.txt
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1340104 (1.3M) [text/plain]
Saving to: ‘16847-0.txt.2’

16847-0.txt.2       100%[===================>]   1.28M  3.24MB/s    in 0.4s    

2022-01-17 17:47:15 (3.24 MB/s) - ‘16847-0.txt.2’ saved [1340104/1340104]



In [110]:
!wget "https://www.gutenberg.org/files/18762/18762-0.txt"

--2022-01-17 17:47:16--  https://www.gutenberg.org/files/18762/18762-0.txt
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1271852 (1.2M) [text/plain]
Saving to: ‘18762-0.txt.2’

18762-0.txt.2       100%[===================>]   1.21M  3.08MB/s    in 0.4s    

2022-01-17 17:47:17 (3.08 MB/s) - ‘18762-0.txt.2’ saved [1271852/1271852]



In [111]:
!pip install unidecode

In [112]:
import codecs
from bs4 import BeautifulSoup
import unidecode

In [113]:
list_books = ['14860-0', '11100-0', '14140-0', '16847-0', '18762-0']
text_add = ''
for b in list_books:
  with open(b + '.txt', 'r') as f:
    text = f.read()

    info_beg = text.find('Title: ')
    begin = text.find('*** START OF THIS PROJECT GUTENBERG EBOOK ')
    title = ''
    if begin > -1:
      for i in text[info_beg : begin].split('\n'):
        if i.find('Title: ') > -1:
          title = i.split('Title: ')[1]
          begin_2 = begin + 42 + len(title) + 3
                
    else:
      begin = text.find('START OF THE PROJECT GUTENBERG EBOOK ')
      for i in text[info_beg : begin].split('\n'):
        if i.find('Title: ') > -1:
          title = i.split('Title: ')[1]
          begin_2 = begin + 37 + len(title) + 3

    end = text.find('*** END OF THIS PROJECT GUTENBERG EBOOK')
    if end == -1:
      end = text.find('*** END')
    text = text[begin_2 : end]

    #remove html tags from text
    soup = BeautifulSoup(text, "html.parser")
    text = soup.get_text(separator=" ")
    
    #remove accented characters from text, e.g. café
    text = unidecode.unidecode(text)

    text_add = text_add + text + '\n'

In [114]:
len(text.split(' '))

290243

In [115]:
# Create two dictionaries:
# int2char -- maps integers to characters
# char2int -- maps characters to unique integers

int2char = dict(enumerate(set(text)))
char2int = { val: idx  for idx, val in int2char.items()}

# encode the text
encoded = np.array([char2int[ch] for ch in text])

In [116]:
len(char2int)

87

In [117]:
def one_hot_encode(arr, n_labels):
    
    # arr - array of integers
    # n_labels - number of labels (the size of a one-hot-encoded vector)

    one_hot = np.eye(n_labels)[arr]
    
    return one_hot

In [118]:
# check that the function works correctly
test_indx = np.array([[7, 2, 5]])
one_hot = one_hot_encode(test_indx, 8)

print(one_hot)

[[[0. 0. 0. 0. 0. 0. 0. 1.]
  [0. 0. 1. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0. 0.]]]


In [119]:
def get_batches(arr, batch_size, seq_length):
    # Create a generator that returns batches of size batch_size x seq_length
    
    batch_size_total = batch_size * seq_length
    ## Get the number of batches we can make
    n_batches = int(len(arr) / batch_size_total)
    
    ## Keep only enough characters to make full batches
    arr = arr[: n_batches*batch_size_total] 
    
    ## Reshape into batch_size rows
    arr = arr.reshape((batch_size, -1)) 
    
    ## Iterate over the batches using a window of size seq_length
    for n in range(0, arr.shape[1], seq_length):
        x = arr[:, n : n + seq_length ]
        # The target is a version of x shifted by one (do not forget border conditions)
        y = np.zeros_like(x)
        y[:, :-1] = x[:, 1:]
        if n + seq_length < arr.shape[1]:
            y[:, -1] = arr[:, n + seq_length]
        else:
            arr[:, 0]
        yield x, y

In [120]:
batches = get_batches(encoded, 4, 30)
x, y = next(batches)

In [121]:
# check if GPU is available
train_on_gpu = torch.cuda.is_available()
if(train_on_gpu):
    print('Training on GPU')
else: 
    print('No GPU available, training on CPU; consider making n_epochs very small.')

Training on GPU


In [122]:
class CharRNN(nn.Module):
    
    def __init__(self, tokens, n_hidden=256, n_layers=2,
                               drop_prob=0.5, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        
        # creating character dictionaries
        self.chars = tokens
        self.int2char = dict(enumerate(self.chars))
        self.char2int = {ch: ii for ii, ch in self.int2char.items()}
        
        # Define the LSTM layer
        ## YOUR CODE HERE
        self.lstm = nn.LSTM(input_size=len(self.chars), hidden_size=self.n_hidden, 
                            num_layers=self.n_layers, dropout=self.drop_prob, 
                            batch_first=True)

        # Define a dropout layer
        ## YOUR CODE HERE
        self.dropout = nn.Dropout(self.drop_prob)

        # Define the final, fully-connected output layer
        ## YOUR CODE HERE
        self.linear = nn.Linear(in_features=self.n_hidden, out_features=len(self.chars))
      
    
    def forward(self, x, hidden):
        ''' Forward pass through the network. 
            These inputs are x, and the hidden/cell state `hidden`. '''
                
        # Get the outputs and the new hidden state from the lstm
        ## YOUR CODE HERE
        out, hidden = self.lstm(x.float(), hidden)

        # Pass through a dropout layer
        ## YOUR CODE HERE
        out = self.dropout(out)

        # Stack up LSTM outputs using view
        # you may need to use contiguous to reshape the output
        ## YOUR CODE HERE
        out = out.reshape(out.size()[0] * out.size()[1], self.n_hidden)


        # Put x through the fully-connected layer
        ## YOUR CODE HERE
        out = self.linear(out)

        # return the final output and the hidden state
        return out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                      weight.new(self.n_layers, batch_size, self.n_hidden).zero_())
        
        return hidden    

In [123]:
def train(net, data, epochs=10, batch_size=10, seq_length=50, lr=0.001, clip=5, val_frac=0.1, print_every=10):
    ''' Training a network 
    
        Arguments
        ---------
        lr: learning rate
        clip: gradient clipping
        val_frac: Fraction of data to hold out for validation
        print_every: Number of steps for printing training and validation loss
    
    '''
    net.train()
    
    opt = torch.optim.Adam(net.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    # create training and validation data
    val_idx = int(len(data)*(1-val_frac))
    data, val_data = data[:val_idx], data[val_idx:]
    
    if(train_on_gpu):
        net.cuda()
    
    counter = 0
    n_chars = len(net.chars)
    for e in range(epochs):
        # initialize hidden state
        h = net.init_hidden(batch_size)
        
        for x, y in get_batches(data, batch_size, seq_length):
            counter += 1
            
            # One-hot encode our data and make them Torch tensors
            x = one_hot_encode(x, n_chars)
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)
            
            if(train_on_gpu):
                inputs, targets = inputs.cuda(), targets.cuda()

            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            h = tuple([each.data for each in h])

            net.zero_grad()
            
            output, h = net(inputs, h)
            
            loss = criterion(output, targets.view(batch_size*seq_length).long())
            loss.backward()

            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs
            nn.utils.clip_grad_norm_(net.parameters(), clip)
            opt.step()
            
            # loss stats
            if counter % print_every == 0:
                # Get validation loss
                val_h = net.init_hidden(batch_size)
                val_losses = []
                net.eval()
                for x, y in get_batches(val_data, batch_size, seq_length):
                    # One-hot encode our data and make them Torch tensors
                    x = one_hot_encode(x, n_chars)
                    x, y = torch.from_numpy(x), torch.from_numpy(y)
                    
                    # Creating new variables for the hidden state, otherwise
                    # we'd backprop through the entire training history
                    val_h = tuple([each.data for each in val_h])
                    
                    inputs, targets = x, y
                    if(train_on_gpu):
                        inputs, targets = inputs.cuda(), targets.cuda()

                    output, val_h = net(inputs, val_h)
                    val_loss = criterion(output, targets.view(batch_size*seq_length).long())
                
                    val_losses.append(val_loss.item())
                
                net.train() # reset to train mode after iterationg through validation data
                now = datetime.now()
                dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.4f}...".format(loss.item()),
                      "Val Loss: {:.4f}".format(np.mean(val_losses)),
                      "Time: ", dt_string)

In [124]:
n_hidden = 512 
n_layers = 2

chars = tuple(set(text))
net = CharRNN(chars, n_hidden, n_layers)
print(net)

CharRNN(
  (lstm): LSTM(87, 512, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (linear): Linear(in_features=512, out_features=87, bias=True)
)


In [125]:
with open('/content/drive/MyDrive/2022-01-15_Course_project/LSTM_generator/dict_chars.pickle', 'wb') as f:
    pickle.dump(chars, f)

In [126]:
with open('/content/drive/MyDrive/2022-01-15_Course_project/LSTM_generator/dict_chars.pickle', 'rb') as f:
  c = pickle.load(f)

In [127]:
print(c)

(';', 'J', 'E', '>', 'S', 'V', 'e', '6', 'R', 'K', '~', 'P', 'H', 'I', '*', ')', 'p', 'l', 'n', '0', '_', '/', 'F', 'k', '<', '"', '?', ' ', 'A', 'O', 'T', 'g', 'W', 'L', "'", 'h', 'm', 'r', 'y', 'x', 's', '[', 'c', 'Z', 'q', '^', 'v', '\\', 'i', 'w', ']', '2', 'B', 'Y', 't', 'o', 'j', '-', 'N', '7', '(', '8', 'G', '.', '1', 'u', '\n', 'D', '4', 'U', '3', 'f', ':', 'd', 'Q', 'a', 'b', 'C', ',', 'M', '{', '9', 'z', '!', 'X', '}', '5')


In [128]:
batch_size = 256
seq_length = 100 
n_epochs = 50

# train the model
train(net, encoded, epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.001, print_every=500)

Epoch: 12/50... Step: 500... Loss: 1.9644... Val Loss: 1.8510 Time:  17/01/2022 17:50:18
Epoch: 24/50... Step: 1000... Loss: 1.6966... Val Loss: 1.5915 Time:  17/01/2022 17:51:07
Epoch: 36/50... Step: 1500... Loss: 1.5369... Val Loss: 1.4719 Time:  17/01/2022 17:51:56
Epoch: 48/50... Step: 2000... Loss: 1.4564... Val Loss: 1.4141 Time:  17/01/2022 17:52:45


In [129]:
path = "/content/drive/MyDrive/2022-01-15_Course_project/LSTM_generator/text_generator.pt" 
torch.save(net.state_dict(), path)


In [130]:
path = "/content/drive/MyDrive/2022-01-15_Course_project/LSTM_generator/text_generator.pt" 


with open('/content/drive/MyDrive/2022-01-15_Course_project/LSTM_generator/dict_chars.pickle', 'rb') as f:
    chars_2 = pickle.load(f)

n_hidden = 512  
n_layers = 2
net_2 = CharRNN(chars_2, n_hidden, n_layers)
net_2.load_state_dict(torch.load(path))

<All keys matched successfully>

In [131]:
net_2.eval()

CharRNN(
  (lstm): LSTM(87, 512, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (linear): Linear(in_features=512, out_features=87, bias=True)
)

In [132]:
def predict(net, char, h=None, top_k=None):
        ''' Given a character, predict the next character.
            Returns the predicted character and the hidden state.
        '''
        
        # tensor inputs
        x = np.array([[net.char2int[char]]])
        x = one_hot_encode(x, len(net.chars))
        inputs = torch.from_numpy(x)
        
        if(train_on_gpu):
            inputs = inputs.cuda()
        
        # detach hidden state from history
        h = tuple([each.data for each in h])
        # get the output of the model
        out, h = net(inputs, h)

        # get the character probabilities
        p = F.softmax(out, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
        
        # get top characters
        if top_k is None:
            top_ch = np.arange(len(net.chars))
        else:
            p, top_ch = p.topk(top_k)
            top_ch = top_ch.numpy().squeeze()
        
        # select the likely next character with some element of randomness
        p = p.numpy().squeeze()
        char = np.random.choice(top_ch, p=p/p.sum())
        
        # return the encoded value of the predicted char and the hidden state
        return net.int2char[char], h

In [133]:
def sample(net, size, prime='The', top_k=None):
        
    if(train_on_gpu):
        net.cuda()
    else:
        net.cpu()
    
    net.eval() # eval mode
    
    # First off, run through the prime characters
    chars = [ch for ch in prime]
    h = net.init_hidden(1)
    for ch in prime:
        char, h = predict(net, ch, h, top_k=top_k)

    chars.append(char)
    
    # Now pass in the previous character and get a new one
    for ii in range(size):
        char, h = predict(net, chars[-1], h, top_k=top_k)
        chars.append(char)

    return ''.join(chars)

In [134]:
# Example of generated text
print(sample(net, 8000, prime='Now pass in the previous character and get a new one', top_k=20))

Now pass in the previous character and get a new onesman,
or all, nor somewwere suvery pirlut to murrey miyd you renuties; on
the sunce less would betroute the rantrom did the dame
(in _Hosry_, his vioter was sturn to streected of an centrementon
"to Mosea still,--but you flowning the let of through"
and among but the land and in my dray, etc., and he proposed to inceward all
piercesion that to that he had not called in a momation complate, thear souls in the
scinistime with what an irpentation to the buck and roveless of;
if you cantalu the red incompased in this scaller's im many
appelition, and in the forgance and by a for her soul's cime
to disis a temole erople of the sweet in 1826; she say the
_true flight decore_ (_Machin_, 1818, p. 405), and Counte,
in 1858, old Accion, Fasting Mays, 1, 28, note 1.]

{498}[mv]
[---- _'ave eney in save_ the man's hels.

                      X.

    Juan, and whring, and dreads the fall-guind wonder
      The world as stoption, how it of much sh

In [135]:
text_ph = text.split('. ')

In [138]:
list_b1 = [ i.split(' ')[0].lower() for i in text_ph]
list_b2 = [ ' '.join(j.lower() for j in i.split(' ')[: 2]) for i in text_ph]
list_b3 = [ ' '.join(j.lower() for j in i.split(' ')[: 3]) for i in text_ph]
list_b4 = [ ' '.join(j.lower() for j in i.split(' ')[: 4]) for i in text_ph]

In [139]:
Counter(list_b1).most_common(10)

[('the', 115),
 ('sc', 74),
 ('he', 69),
 ('i', 58),
 ('line', 48),
 ('it', 39),
 ('in', 38),
 ('a', 37),
 ('lines', 35),
 ('de', 33)]

In [140]:
import re

In [141]:
list_of_begins = set()
for i in Counter(list_b1).most_common(10):
  list_of_begins.add(re.sub("[^A-Za-z] ", "", i[0]).capitalize())

for i in Counter(list_b2).most_common(10):
  list_of_begins.add(re.sub("[^A-Za-z] ", "", i[0]).capitalize())

for i in Counter(list_b3).most_common(10):
  list_of_begins.add(re.sub("[^A-Za-z] ", "", i[0]).capitalize())

In [142]:
list(list_of_begins)

['He was',
 'De',
 'In',
 'The',
 'A',
 'Pp',
 'P',
 'I',
 'It is',
 'Ii',
 'He',
 'De la',
 'De la nouvelle',
 'Ed',
 'Lines',
 'Line',
 'Sc',
 'Vi',
 'It',
 'Cap',
 'Seetoo,']

In [143]:
import random

In [ ]:
num_docs = 15000

path = '/content/drive/MyDrive/2022-01-15_Course_project/gen_text_en/'

for i in range(num_docs):
  start = random.choice(list(list_of_begins))
  text_gen = sample(net, 8000, prime= start, top_k=10)

  with open(path + 'doc_'+str(i)+'.txt', 'w+') as output_file:
    try:
      output_file.write(text_gen)
    except:
      continue
  if i % 200 == 0:
    now = datetime.now()
    dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
    print(dt_string, '| ',  'number of processed files: ' + str(i), round(i/num_docs, 3)*100, '%')


17/01/2022 10:52:14 |  number of processed files: 0 0.0 %
17/01/2022 11:05:44 |  number of processed files: 200 1.3 %
17/01/2022 11:19:12 |  number of processed files: 400 2.7 %
17/01/2022 11:32:24 |  number of processed files: 600 4.0 %
17/01/2022 11:45:38 |  number of processed files: 800 5.3 %
17/01/2022 11:58:52 |  number of processed files: 1000 6.7 %
17/01/2022 12:12:09 |  number of processed files: 1200 8.0 %
17/01/2022 12:25:24 |  number of processed files: 1400 9.3 %
17/01/2022 12:38:43 |  number of processed files: 1600 10.7 %
17/01/2022 12:52:02 |  number of processed files: 1800 12.0 %
17/01/2022 13:05:22 |  number of processed files: 2000 13.3 %
17/01/2022 13:18:42 |  number of processed files: 2200 14.7 %
17/01/2022 13:32:08 |  number of processed files: 2400 16.0 %
17/01/2022 13:45:32 |  number of processed files: 2600 17.299999999999997 %
17/01/2022 13:58:44 |  number of processed files: 2800 18.7 %
17/01/2022 14:11:58 |  number of processed files: 3000 20.0 %
17/01/202

In [144]:
# If the generator does not work completely, you need to run this code (it will start from the last place)

# This is the code to generate (to determine how many have already been generated)

list_gen = glob.glob('/content/drive/MyDrive/2022-01-15_Course_project/gen_text_en/*.txt')
list_num = [ int(i.split('/')[-1].split('.txt')[0].split('doc_')[-1]) for i in list_gen]
max(list_num), min(list_num)


(5446, 0)

In [145]:
# If the generator does not work completely, you need to run this code (it will start from the last place)

num_docs = 15000

path = '/content/drive/MyDrive/2022-01-15_Course_project/gen_text_en/'

for i in range(max(list_num), num_docs, 1):
  start = random.choice(list(list_of_begins))
  text_gen = sample(net, 8000, prime= start, top_k=10)

  with open(path + 'doc_'+str(i)+'.txt', 'w+') as output_file:
    try:
      output_file.write(text_gen)
    except:
      continue
  if i % 200 == 0:
    now = datetime.now()
    dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
    print(dt_string, '| ',  'number of processed files: ' + str(i), round(i/num_docs, 3)*100, '%')

17/01/2022 18:10:43 |  number of processed files: 5600 37.3 %
17/01/2022 18:24:18 |  number of processed files: 5800 38.7 %
17/01/2022 18:37:54 |  number of processed files: 6000 40.0 %
17/01/2022 18:51:32 |  number of processed files: 6200 41.3 %
17/01/2022 19:05:10 |  number of processed files: 6400 42.699999999999996 %
17/01/2022 19:18:48 |  number of processed files: 6600 44.0 %
17/01/2022 19:32:26 |  number of processed files: 6800 45.300000000000004 %
17/01/2022 19:46:03 |  number of processed files: 7000 46.7 %
17/01/2022 19:59:42 |  number of processed files: 7200 48.0 %
17/01/2022 20:13:20 |  number of processed files: 7400 49.3 %
17/01/2022 20:26:59 |  number of processed files: 7600 50.7 %
17/01/2022 20:40:37 |  number of processed files: 7800 52.0 %
17/01/2022 20:54:16 |  number of processed files: 8000 53.300000000000004 %
17/01/2022 21:07:56 |  number of processed files: 8200 54.7 %
17/01/2022 21:21:36 |  number of processed files: 8400 56.00000000000001 %
17/01/2022 21:3